In [2]:
import tensorflow as tf
from tensorflow.keras.layers import Dense,LeakyReLU,BatchNormalization,Reshape,Flatten
from tensorflow.keras.models import Sequential
import numpy as np
import matplotlib.pyplot as plt

In [3]:
def build_generator():
    model=Sequential()
    model.add(Dense(128,input_dim=100))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization())
    model.add(Dense(784,activation='tanh'))
    model.add(Reshape((28,28)))
    return model

In [4]:
def build_discriminator():
    model=Sequential()
    model.add(Flatten(input_shape=(28,28)))
    model.add(Dense(128))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(1,activation='sigmoid'))
    return model

In [5]:
(X_train,_),_=tf.keras.datasets.mnist.load_data()
X_train=(X_train.astype(np.float32)-127.5)/127.5

In [6]:
generator = build_generator()
discriminator = build_discriminator()

# Compile Discriminator
discriminator.compile(optimizer='adam', loss='binary_crossentropy')

c:\Users\appam\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
c:\Users\appam\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\activations\leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(
c:\Users\appam\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [7]:
z = tf.keras.Input(shape=(100,))
img=generator(z)
discriminator.trainable=False
validity=discriminator(img)
gan = tf.keras.Model(inputs=z, outputs=validity)
gan.compile(optimizer='adam', loss='binary_crossentropy')

In [ ]:
for epoch in range(10000):
    idx = np.random.randint(0, X_train.shape[0], 64)
    real_images = X_train[idx]
    noise = np.random.normal(0, 1, (64, 100))
    fake_images=generator.predict(noise)
    discriminator.trainable= True
    discriminator.train_on_batch(real_images, np.ones((64, 1)))
    discriminator.train_on_batch(fake_images, np.zeros((64, 1)))
    discriminator.trainable = False
    gan.train_on_batch(noise,np.ones((64,1)))
    if epoch%200==0:
        plt.imshow(fake_images[0],cmap='gray')
        plt.title(f"Epoch {epoch}")
        plt.show()

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step 
